# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(mailto = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 500,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&obj-id.prefix=10.1101&source=twitter&rows=500&from-occurred-date=2021-11-01&until-occurred-date=2021-11-08


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

13300 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(mailto = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-11-01&until-occurred-date=2021-11-08
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=16c3dcc4-1a9d-45c6-8b3d-b9541a8e005a&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-11-01&until-occurred-date=2021-11-08
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&cursor=dd28598a-ae9c-4306-bcb7-069e34a99f3c&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-11-01&until-occurred-date=2021-11-08
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=info@front-matter.io&curso

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-10-18.csv
failed to load preprint_tweets_2021-10-25.csv
failed to load preprint_tweets_2021-07-29.csv
failed to load preprint_tweets_2021-09-06.csv
failed to load preprint_tweets_2021-09-13.csv
failed to load .DS_Store
failed to load preprint_tweets_2021-10-23.csv
failed to load .gitkeep
failed to load preprint_tweets_2021-08-30.csv
failed to load preprint_tweets_2021-11-01.csv
failed to load preprint_tweets_2021-08-09.csv
failed to load preprint_tweets_2021-08-23.csv
failed to load preprint_tweets_2021-08-02.csv
failed to load preprint_tweets_2021-08-16.csv
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-10-05.csv
failed to load preprint_tweets_2021-10-11.csv
failed to load preprint_tweets_2021-10-04.csv
failed to load preprint_tweets_2021-10-06.csv
failed to load preprint_tweets_2021-10-07.csv
failed to load preprint_tweets_2021-07-26.csv
failed to load preprint_tweets_2021-09-20.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.08.24.21262415...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264262...
REST API query complete  200
REST API query started for 10.1101/2021.09.13.21262182...
REST API query complete  200
REST API query started for 10.1101/2021.09.22.21263977...
REST API query complete  200
REST API query started for 10.1101/2021.09.28.21264260...
REST API query complete  200
REST API query started for 10.1101/2021.02.16.21251535...
REST API query complete  200
REST API query started for 10.1101/2021.07.31.21261387...
REST API query complete  200
REST API query started for 10.1101/2021.09.16.21263684...
REST API query complete  200
REST API query started for 10.1101/2021.05.03.21256520...
REST API query complete  200
REST API query started for 10.1101/2021.09.30.462488...
REST API query complete  200
REST API query started for 10.1101/2021.06.11.21258690...
REST API query complete  200
REST API query started for 10.1101/2021.07.07

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.11.05.467528,307,bioRxiv,Bioengineering,False,Large-scale discovery of recombinases for inte...,"[{'name': 'Matthew G Durrant'}, {'name': 'Alis...",<p>Recent microbial genome sequencing efforts ...,2021-11-06
1,10.1101/2021.10.31.21265703,176,medRxiv,Infectious Diseases (except HIV/AIDS),True,"Efficacy and safety of SOBERANA 02, a COVID-19...","[{'name': 'Maria Eugenia Toledo-Romani'}, {'na...",<p>Background: SOBERANA 02 is a COVID19 conjug...,2021-11-01
2,10.1101/2021.11.03.467152,146,bioRxiv,Microbiology,False,DeepBacs: Bacterial image analysis using open-...,"[{'name': 'Christoph Spahn'}, {'name': 'Romain...",<p>Deep Learning (DL) is rapidly changing the ...,2021-11-03
3,10.1101/2021.11.02.466936,113,bioRxiv,Molecular Biology,False,"Bright, fluorogenic and photostable avidity pr...","[{'name': 'Bastian Bühler'}, {'name': 'Anja Be...",<p>Fluorescent light-up aptamers (FLAPs) emerg...,2021-11-02
4,10.1101/2021.11.02.466983,95,bioRxiv,Biophysics,False,Structure Determination of Inactive-State GPCR...,"[{'name': 'Michael J. Robertson'}, {'name': 'F...",<p>Cryogenic electron microscopy (cryo-EM) has...,2021-11-04
5,10.1101/2021.11.01.466765,88,bioRxiv,Evolutionary Biology,False,Global patterns and rates of habitat transitio...,"[{'name': 'Mahwash Jamy'}, {'name': 'Charlie B...",<p>The successful colonisation of new habitats...,2021-11-03
6,10.1101/2021.11.03.467131,68,bioRxiv,Biochemistry,False,A molecular network of conserved factors keeps...,"[{'name': 'Friederike Leesch'}, {'name': 'Laur...",<p>Ribosomes are produced in large quantities ...,2021-11-03
7,10.1101/2021.11.04.467077,68,bioRxiv,Microbiology,True,The oral protease inhibitor (PF-07321332) prot...,"[{'name': 'Rana Abdelnabi'}, {'name': 'Carolin...",<p>There is an urgent need for potent and sele...,2021-11-05
8,10.1101/2021.11.02.466915,58,bioRxiv,Molecular Biology,False,RBBP6 activates the pre-mRNA 3’-end processing...,"[{'name': 'Vytaute Boreikaite'}, {'name': 'Tho...",<p>3’-end processing of most human mRNAs is ca...,2021-11-02
9,10.1101/2021.11.03.467079,53,bioRxiv,Genomics,False,Parent-of-origin effects in the UK Biobank,"[{'name': 'Robin J Hofmeister'}, {'name': 'Sim...",<p>Identical genetic variations can have diffe...,2021-11-06


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

237 preprints (including 40 covering SARS-CoV-2, 200 from bioRxiv and 37 from medRxiv) published in the last 7 days before 2021-11-08 had been tweeted at least 3 times (maximum 307).